## Problem 1
Write a recursive function to find a number in an unordered list and return the index.
Assume that the number appears only once.

A recursive function might not be the best choice to search an unordered list. We are using it for a warm up exercise.

In [92]:
def number_search(num_list=[], num_find=0 ):
    """ 
    Returns the index where num_find was found in num_list 
    Return -1 if the number is not in the list
    
    Arguments: 
    num_list
    num_find
    Returns:
    num_index
    """
    ### DO NOT CHANGE THE FUNCTION NAME OR ARGUMENTS ###
    ### IT WILL BREAK THE TEST SCRIPT###
    index = -1
    
    #NEED TO CREATE A VARIABLE TO STORE EACH VALUE IN A LIST
    #global index
    if num_list == []:
        return -1
    elif num_list[0] == num_find:
        return 0
    
    index = number_search(num_list[1:], num_find)
    
    if index < 0:
        return index
    
    return index + 1
    ## End your code
    
    return index
    
# Use this portion if you want to add unit tests to validate your code
number_search(num_list = [6,9,1,0,-12,8], num_find= 0)



3

In [ ]:
# Test function
def test1(num_list,num_find):
    inx_val = -1
    if num_find in num_list:
        inx_val = num_list.index(num_find)
    if number_search(num_list,num_find) == inx_val:
           return True
    else:
           return False

    

num_list = [6,9,1,0,-12,8]
num_find = 9
if test1(num_list,num_find):
    print("Test 1 worked on list {} and searching for {}".format(num_list,num_find))
else:
    print("Test 1 FAILED on list {} and searching for {}".format(num_list,num_find))
num_list = [6,9,1,0,-12,8]
num_find = 10
if test1(num_list,num_find):
    print("Test 1 worked on list {} and searching for {}".format(num_list,num_find))
else:
    print("Test 1 FAILED on list {} and searching for {}".format(num_list,num_find))


## Problem 2
Write a function to solve a number puzzle.
I will use a "0" to represent the empty square.



In [94]:
import random
import math
import copy
from copy import deepcopy
import datetime as dt
import sys
import numpy as np
 
    
class EightPuzzle(object):
    def __init__(self,board=[],debug=False):
        self._parent = None
        if board != []:
            self.board = board
        else:
            self.generate_random()
        self.debug = debug
        self._depth = 0
        self.solved_board = [[0,1,2],[3,4,5],[6,7,8]]
    
    def generate_random(self):
        """ This function generates a random 3x3 board 
        it is useful for testing
        """
        
        valid_board = False
        self.board = []
        while not valid_board:
            entries = [x for x in range(9)]
            random.shuffle(entries)
            for i in range(3):
                line =[]
                for j in range(3):
                    line.append(entries.pop())
                self.board.append(line)
            if  self.isSolvable():
                valid_board = True
            else:
                self.board = []
            
    
    ### TO DO - LOOP UNTIL YOU HAVE A SOLVABLE BOARD
    
    def _swap_and_clone(self,a):
        """
        This is borrowed from the same around as _clone
        It will is used when you make a move. 
        You create a new board then swap the items in the move
        
        Since I will only ever swap with 0, I probably should just find 0 as part of this
        """
        b = self.find(0)
        p = copy.deepcopy(self)
        p.swap(a,b)
        p._depth = self._depth + 1
        p._parent = self
        return p
    
    def swap(self,pos_a,pos_b):
        """
        Basic swap function
        """
        temp = self.board[pos_a[0]][pos_a[1]]
        self.board[pos_a[0]][pos_a[1]] = self.board[pos_b[0]][pos_b[1]]
        self.board[pos_b[0]][pos_b[1]] = temp
    
    def check_solved(self,board=None):
        """ 
        This is a trivial function that just looks if the two boards are the same
        It returns true if the solved board matches the current board
        """
        if board == None:
            board = self.board
        return self.solved_board == board
    
    def find(self, value):
        """returns the row, col coordinates of the specified value
           on the board """
        
        if value < 0 or value > len(self.board) ** 2:
            raise Exception("value out of range")

        for row in range(len(self.board)):
            for col in range(len(self.board)):
                if self.board[row][col] == value:
                    return row, col     
        
    def legal_moves(self):
        """ 
        This will return a list of tuples that are adjacent to the free square
        In our case, the free square is represented as a 0
        """

        blank_row, blank_col = self.find(0)
        peek_in = []
        if blank_row >= 0 and blank_row < (len(self.board) - 1):
            # Look below
            peek_in.append([blank_row+1,blank_col])
        if blank_row > 0 and blank_row < (len(self.board) ):
            # Look above
            peek_in.append([blank_row-1,blank_col])
        if blank_col >= 0 and blank_col < (len(self.board) - 1):
            # Look right
            peek_in.append([blank_row,blank_col+1])
        if blank_col > 0 and blank_col < (len(self.board) ):
            # Look left
            peek_in.append([blank_row,blank_col-1])
        
        return peek_in
    
    def print_board(self,board):
        for row in range(len(board)):
            print("-"*(4*len(board)+2))
            print("|",end=' ')
            for col in range(len(board)):
                if board[row][col] == 0:
                    print(' ',end=" | ")
                else:
                    print(board[row][col], end=' | ')
            print()  
        print("-"*(4*len(board)+2))
        
    def print_path(self,path):
        for i in range(len(path)):
            print("\nBoard",i)
            self.print_board(path[i])
        
    def getInvCount(self,arr):
        inv_count = 0
        empty_value = 0
        for i in range(0, 9):
            for j in range(i + 1, 9):
                if arr[j] != empty_value and arr[i] != empty_value and arr[i] > arr[j]:
                    inv_count += 1
        return inv_count
 
     
    # This function returns true
    # if given 8 puzzle is solvable.
    
    def isSolvable(self) :
        # Count inversions in given 8 puzzle
        inv_count = self.getInvCount([j for sub in self.board for j in sub])
        # return true if inversion count is even.
        return (inv_count % 2 == 0)
    

### Functions needed for Solution Class.......

    def manhattan_calculation(self,s, g):
        """
        It will calculate Manhattan distance sum for each digit as per given in our goal.
        """
        m = abs(s // 3 - g // 3) + abs(s % 3 - g % 3)
        return sum(m[1:])

    def coordinates(self,s):
        """
        It is basically helper function to the manhattan_calculation function. It assigns each digit the coordinate 
        to calculate Manhattan distance.
        """
        c = np.array(range(9))
        for x, y in enumerate(s):
            c[y] = x
        return c

    def inversions(self,s):
        """
        It helps to check if the initial state is solvable via inversion calculation.
        """
        k = s[s != 0]
        tinv = 0
        for i in range(len(k) - 1):
            b = np.array(np.where(k[i+1:] < k[i])).reshape(-1)
            tinv += len(b)
        return tinv

    def optimized_steps(self,state):
        """
        It generates board list as per optimized steps in sequence.
        """
        optimal = np.array([], int).reshape(-1, 9)
        last = len(state) - 1
        while last != -1:
            optimal = np.insert(optimal, 0, state[last]['board'], 0)
            last = int(state[last]['parent'])
        return optimal.reshape(-1, 3, 3)

### Solution 
Put everything you do in the Solution subclass.


In [96]:
class Solution(EightPuzzle):
    
    def __init__(self,board=[]):
        # Constructor for my subclass
        super().__init__(board)
        
        """ This subclass is almost all yours
        The only restriction is that you need to have a solution() method which will return the solution path
        I recommend that you treat it mostly like a dummy function and just call your function, but it is up to you
        """
        
    def puzzle_board(self,board, goal):
        """
        This function solves the board using A* and manhattan distance.
        """
    
        moves = np.array([('u', [0, 1, 2], -3),    # These are the legal moves possible in 8-number puzzle problem.
                          ('d', [6, 7, 8],  3),
                          ('l', [0, 3, 6], -1),
                          ('r', [2, 5, 8],  1)],
                    dtype=  [('move',  str, 1),
                            ('pos',   list),
                            ('delta', int)]
                        )

        dtstate = [ ('board',  list),
                    ('parent', int),
                    ('gn',     int),
                    ('hn',     int)
                    ]

        goalc = self.coordinates(goal)
        # initial state values
        parent = -1
        gn     = 0
        hn     = self.manhattan_calculation(self.coordinates(board), goalc)  # Calling manhattan function.
        state = np.array([(board, parent, gn, hn)], dtstate)

        #priority queue initialization
        dtpriority = [  ('pos', int),
                        ('fn', int)
                        ]

        priority = np.array( [(0, hn)], dtpriority)

        while True:
            priority = np.sort(priority, kind='mergesort', order=['fn', 'pos']) # sort priority queue
            pos, fn = priority[0]                   # pick out first from sorted to explore
            priority = np.delete(priority, 0, 0)    # remove from queue what we are exploring
            board, parent, gn, hn = state[pos]
            board = np.array(board)
            loc = int(np.where(board == 0)[0])      # locate '0' (blank)
            gn = gn + 1                             # increase cost g(n) by 1

            for m in moves:
                if loc not in m['pos']:
                    succ = deepcopy(board)          # generate new state as copy of current
                    succ[loc], succ[loc + m['delta']] = succ[loc + m['delta']], succ[loc]   # do the move

                    if ~(np.all(list(state['board']) == succ, 1)).any():    # check if new (not repeat)
                        hn = self.manhattan_calculation(self.coordinates(succ), goalc) # calculate Manhattan distance
                        q = np.array(   [(succ, pos, gn, hn)], dtstate)     # generate and add new state in the list
                        state = np.append(state, q, 0)
                        fn = gn + hn                                        # calculate f(n)
                        q = np.array([(len(state) - 1, fn)], dtpriority)    # add to priority queue
                        priority = np.append(priority, q, 0)

                        if np.array_equal(succ, goal):                      # Checking is this goal state or not?
                            return state, len(priority)

        return state, len(priority)        
    
    def solution(self):
        """ 
        Once completed this should return your puzzle path
        If having puzzle_path = [] as the first line of your funciton doesn't work for you, it is ok to remove it
        """
        
        goal = []      # Coverting our goal value stored in self.solved_baord to simple list value for easy access.
        for i in self.solved_board:
            for j in i:
                goal.append(j)
        
        goal = np.array(goal)
        given_board = "" 

        for i in self.board:     # To convert the board in the simple string for easy access.
            for j in i:
                given_board=given_board+str(j) 
        string = given_board

        board = np.array(list(map(int, string)))
        if (self.inversions(board) % 2 != 0):    ## If the board is unsolvable...
            print('YOU GAVE ME A BAD PUZZLE!! PLEASE RUN THE CELL AGAIN UNTIL YOU GET SOLVABLE PUZZLE.')
            return

        state, explored = self.puzzle_board(board, goal)
        # generate and show optimized steps
        optimal = self.optimized_steps(state) #Total optimized steps or path_length
        return optimal

In [97]:
# This is my validation function for problem 2
def problem2_checker():
    points = 0
    try:
        print("Testing with a simple board")
        puzzle = Solution(board=[[1,2,0],[3,4,5],[6,7,8]])
        puzzle_path = puzzle.solution()
        if 1 < len(puzzle_path) <= 4 : 
            print("Your solution to the trivial puzzle has 4 or fewer in the path. +20 points ")
            points += 20
        else:
            print("You got a solution to the trivial puzzle. +10 points")
            points += 10
        puzzle = Solution(board=[[4,2,0],[3,1,5],[7,6,8]])
        puzzle_path = puzzle.solution()
        if 1 < len(puzzle_path) <= 150 : 
            print("You solved the second puzzle with a path length of {}. +20 points ".format(len(puzzle_path)))
            points += 20
        else:
            print("You got a solution to the second puzzle with a path length of {}. +10 points".format(len(puzzle_path)))
            points += 10
        print("Testing 3 random boards")
        for i in range(2):
            puzzle = Solution(  )
            print("Random board",i)
            puzzle.print_board(puzzle.board)
            puzzle_path = puzzle.solution()
            if 1 < len(puzzle_path) < 10000: # This is ridiculous limit. If the script did crash, you should have this
                 print("Random puzzle {} was solved with a path length of {}. +20 points".format(i+1,len(puzzle_path)))
                 points +=20
    except:
        print("You got an exception. Returning {} points".format(points))
        #raise
        
    return points

total_points = 0
try:
    num_list = [6,9,1,0,-12,8]
    num_find = 9
    if test1(num_list,num_find):
        print("Problem 1 Test 1 worked on list {} and searching for {}".format(num_list,num_find))
        total_points +=10
    else:
        print("Problem 1 Test 1 FAILED on list {} and searching for {}".format(num_list,num_find))
    num_list = [6,9,1,0,-12,8]
    num_find = 10
    if test1(num_list,num_find):
        print("Problem 1 Test 2 worked on list {} and searching for {}".format(num_list,num_find))
        total_points += 10
    else:
        print("Problem 1 Test 2 FAILED on list {} and searching for {}".format(num_list,num_find))   
except:
    print("You hit an exception at {} points.\n I am not handling errors.".format(total_points))
    

total_points += problem2_checker()

print("The auto checker is assigning {} out of 100 points.".format(total_points))
print("If you see a flaw in the auto checker or assignment, email wjwilson@sdsu.edu")


Problem 1 Test 1 worked on list [6, 9, 1, 0, -12, 8] and searching for 9
Problem 1 Test 2 worked on list [6, 9, 1, 0, -12, 8] and searching for 10
Testing with a simple board
Your solution to the trivial puzzle has 4 or fewer in the path. +20 points 
You solved the second puzzle with a path length of 19. +20 points 
Testing 3 random boards
Random board 0
--------------
|   | 1 | 6 | 
--------------
| 5 | 4 | 7 | 
--------------
| 3 | 2 | 8 | 
--------------
Random puzzle 1 was solved with a path length of 27. +20 points
Random board 1
--------------
|   | 3 | 8 | 
--------------
| 4 | 2 | 5 | 
--------------
| 1 | 6 | 7 | 
--------------
Random puzzle 2 was solved with a path length of 19. +20 points
The auto checker is assigning 100 out of 100 points.
If you see a flaw in the auto checker or assignment, email wjwilson@sdsu.edu
